This script computes the CSIM score for different head rotations. 
You need to install deepface library

In [1]:
#you can use following link to install deepface: https://github.com/serengil/deepface
import pandas as pd
import os
from deepface import DeepFace

2023-06-19 18:27:04.836563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Change following paths to match your ground-truth path (ground_truth_parent_directory) and path where the generated frames by your algorithm  are located (generated_parent_directory)

In [2]:
ground_truth_parent_directory = '3D_real_head_dataset/images/ground_truth/'
generated_parent_directory = 'DAGAN_results_3D_heads/'
#generated_parent_directory = 'FOM_results_3D_head/'


In [3]:

#Directories containing ground-truth 
ground_truth_directories = [ground_truth_parent_directory+d for d in os.listdir(ground_truth_parent_directory) 
                            if os.path.isdir(os.path.join(ground_truth_parent_directory, d))]
#Directories containing generated reenactment frames 
generated_directories = [generated_parent_directory+d for d in os.listdir(generated_parent_directory) 
                         if os.path.isdir(os.path.join(generated_parent_directory, d))] 

#For each ground-truth find all deepfake folders that contain the same head rotation.
#ground-truth folder is key and the value is gnerated folders (both contain the same head rotation)
gt_gen_path_dict = {}
for dir_gt in ground_truth_directories:
    rotation_type_gt = dir_gt.split('/')[-1].split('_')[0] #head rotation of ground-truth
    if rotation_type_gt not in gt_gen_path_dict:
                gt_gen_path_dict[dir_gt] = []
    #if the same head rotation is found in generated folder, add it as a value to dict
    for dir_gen in generated_directories: 
        rotation_type_gen = dir_gen.split('/')[-1].split('_')[0]
        
        if rotation_type_gen == rotation_type_gt:
            gt_gen_path_dict[dir_gt].append(dir_gen)

In [9]:
def compute_CSIM(orig_frame, fake_frame):
    """
    Compute csim score between video frames.
    Args:
    orig_frame: ground-truth frame.
    fake_frame: fake frame
    Returns:
    csim score between ground-truth and fake frame
    """
    CSIM_deep_score_cos  = DeepFace.verify(fake_frame, orig_frame, distance_metric = "cosine", enforce_detection=False, 
                                model_name = "ArcFace", detector_backend = 'mtcnn')  
    CSIM_deep_score = 1 - CSIM_deep_score_cos["distance"]
    return CSIM_deep_score


def compute_csim_per_frame(original_path, path1, path2, path3, path4):
    """
    Compute CSIM between ground truth and generated frames for one video (100 frames). We use 4 different 
    identities to generate the same head rotation 
    Args:
    original_path: path to ground-truth directory
    path1, path2, path3, path4: path to deepfake generated directories. Inside each path we have fake frames generated
    by different identities
    Returns:
    A list of CSIM scores between the ground-truth frames and fake frames. The score for each frame is the average
    over the four generated images.
    """
    csim_list = []
    ground_truth_frames_212 = [original_path + '/' + d for d in sorted(os.listdir(original_path))]
    generated_frames_122 = [path1 + '/' + d for d in sorted(os.listdir(path1))]
    generated_frames_340 = [path2 + '/' + d for d in sorted(os.listdir(path2))]
    generated_frames_344 = [path3 + '/' + d for d in sorted(os.listdir(path3))]
    generated_frames_359 = [path4 + '/' + d for d in sorted(os.listdir(path4))]
    print(original_path)
    for ground_truth, gen_122, gen_340, gen_344, gen_359 in zip(ground_truth_frames_212, 
                                                            generated_frames_122, 
                                                            generated_frames_340,
                                                            generated_frames_344,
                                                            generated_frames_359):
        
        CSIM_score_122 = compute_CSIM(ground_truth, gen_122 )
        CSIM_score_340 = compute_CSIM(ground_truth, gen_340)
        CSIM_score_344 = compute_CSIM(ground_truth, gen_344)
        CSIM_score_359= compute_CSIM(ground_truth, gen_359)
        
        mean_csim = (CSIM_score_122+CSIM_score_340+CSIM_score_344+CSIM_score_359)/4 

        csim_list.append(mean_csim)
    return csim_list

In [10]:
def main():
    
    # Create a dataframe to store image name and Csim score for different head rotation per frame
    dataframe = pd.DataFrame({'image_name': []})
    for groundtruth_path, generate_path  in gt_gen_path_dict.items():
        names = [d for d in sorted(os.listdir(groundtruth_path))] #video frame names
        dataframe["image_name"] = names 
        csim_list = []
        rotatio_type = groundtruth_path.split('/')[-1].split('_')[0]
        print(rotatio_type)
        #In the paper we only evaluate the three head rotations below
        if rotatio_type == "pitchNegative": #head rotation of ground-truth
            csim_list = compute_csim_per_frame(groundtruth_path, generate_path[0], generate_path[1], 
                                                generate_path[2], generate_path[3])
            dataframe["pitchNegative"] = csim_list

        if rotatio_type == "yawPositive": #head rotation of ground-truth
            csim_list = compute_csim_per_frame(groundtruth_path, generate_path[0], generate_path[1], 
                                                generate_path[2], generate_path[3])
            dataframe["yawPositive"] = csim_list
            
                    
        if rotatio_type == "pitchYawPositive": #head rotation of ground-truth
            csim_list = compute_csim_per_frame(groundtruth_path, generate_path[0], generate_path[1], 
                                                generate_path[2], generate_path[3])
            dataframe["pitchYawPositive"] = csim_list
    
    print(dataframe)   
    csv_path =   "cv_scores/"+generated_parent_directory
    if not os.path.exists(csv_path):
        os.makedirs(csv_path, exist_ok=True)
                
    dataframe.to_csv(csv_path+"csim_scores_per_frame.csv")  

if __name__ == "__main__":
    main()


pitchNegative
3D_real_head_dataset/images/ground_truth/pitchNegative_id212
1/1 [==============================] - 0s 57ms/step


1/1 [==============================] - 0s 50ms/step
[0.9792410894105573, 0.9568664176507852, 0.9668321139295211, 0.9266144725669805, 0.9199269330059552, 0.9315917419857238, 0.9310369493830086, 0.9181991352891515, 0.9014563327651043, 0.932773867538726, 0.9423859546679327, 0.939923569666002, 0.93344465666879, 0.8969185611651593, 0.8993441591662321, 0.9125482470710334, 0.8761294418397491, 0.9052603289550004, 0.8873165261378626, 0.9122623241710996, 0.8757219924465951, 0.8870530789505687, 0.8867483557984027, 0.8634974194214375, 0.8265988851670161, 0.881510165946904, 0.8577650563376648, 0.8540361645464043, 0.8587872432128245, 0.8471197547802441, 0.8531591991229266, 0.8679654656105994, 0.8049043727098053, 0.8233909359039799, 0.8462692311182141, 0.8490321332624197, 0.8459657249574892, 0.8308432211616615, 0.8026158684314817, 0.7984877188674174, 0.8032812648278155, 0.8530365138410547, 0.8459812413915951, 0.8011541507672316, 0.8097306808793104, 0.7851380300401616, 0.7649376248992102, 0.8134615298